In [1]:
# librerias requeridas
import requests
import os.path
import os
import pandas as pd
import numpy as np
import datetime
import dateutil
import subprocess
import sys
import tempfile
import json
!pip install unidecode
import unidecode
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
!pip install plotly==4.9.0
import plotly.graph_objects as go
from plotly.offline import plot

In [2]:
# muestras procesadas diarias
url_muestras_diarias = 'https://e.infogram.com/api/live/flex/638d656c-c77b-4326-97d3-e50cb410c6ab/8188140c-8352-4994-85e3-2100a4dbd9db?'

In [3]:
# Official Samples Processed Until Now
with requests.get(url_muestras_diarias) as data_muestras:
    with open(os.path.join('../Input_data', 'muestras_procesadas.json'), 'w') as json_file:
        json_data = data_muestras.json()
        del json_data['refreshed']
        json.dump(json_data, json_file, ensure_ascii=False, indent=4)

In [4]:
# Open Official Samples Processed
with open(os.path.join('../Input_data', 'muestras_procesadas.json')) as data_muestras:
    data_muestras = json.load(data_muestras)
# Official Samples Processed
data_muestras = data_muestras['data'][0]
muestras_procesadas_diarias = pd.DataFrame(columns=data_muestras[0], data=data_muestras[1:])

In [5]:
# Update Name Columns
# Remove Accents and Uppercase
muestras_procesadas_diarias.columns = [unidecode.unidecode(value).upper() for value in muestras_procesadas_diarias.columns]

In [6]:
# Setup Date Format
def setup_date_samples(value):
    #print('date:', value)
    try:
        value = value.split(' ')
        value = value[0].split('/')
        #print(len(value))
        if len(value) == 3:
            # Day
            if len(value[0]) == 1:
                value[0] = '0' + value[0]
            # Month
            if len(value[1]) == 1:
                value[1] = '0' + value[1]
            # Year
            if len(value[2]) == 2:
                value[2] = value[2] + '20'
            # Date
            value = value[0] + '/' + value[1] + '/' + value[2]
        else:
            value = '-'
    except IndexError:
        value = '-'
    #print('VALUE:', value)
    if len(value) != 10 and len(value) != 1:
        value = '-'
    return value
# Setup Date Format
muestras_procesadas_diarias['FECHA'] = muestras_procesadas_diarias['FECHA'].transform(lambda value: setup_date_samples(value))
# Show dataframe
muestras_procesadas_diarias.head()

,FECHA,PRUEBAS PROCESADAS POR CORTE
0,-,601
1,05/03/2020,636
2,06/03/2020,739
3,07/03/2020,810
4,08/03/2020,813


In [7]:
# Save dataframe
muestras_procesadas_diarias.to_csv(os.path.join('../Input_data', 'muestras_procesadas.csv'), index=False)

In [51]:
muestras_procesadas = muestras_procesadas_diarias[2:]
muestras_procesadas = [float(x.replace(',','')) for x in muestras_procesadas['PRUEBAS PROCESADAS POR CORTE']]
muestras_procesadas = pd.DataFrame(muestras_procesadas)
muestras_procesadas['PRUEBAS PROCESADAS POR CORTE'] = muestras_procesadas[0]
muestras_procesadas = muestras_procesadas.drop([0], axis=1)
muestras_procesadas

,PRUEBAS PROCESADAS POR CORTE
0,739.0
1,810.0
2,813.0
3,851.0
4,927.0
...,...
178,2625052.0
179,2647702.0
180,2676154.0
181,2699889.0


In [56]:
muestras_procesadas['MUESTRAS_T1']= muestras_procesadas['PRUEBAS PROCESADAS POR CORTE'].shift(1)
muestras_procesadas = muestras_procesadas.dropna()
muestras_procesadas = muestras_procesadas.apply(pd.to_numeric)
muestras_procesadas['PRUEBAS_DIARIAS']= muestras_procesadas['PRUEBAS PROCESADAS POR CORTE']- muestras_procesadas['MUESTRAS_T1']
muestras_procesadas


,PRUEBAS PROCESADAS POR CORTE,MUESTRAS_T1,PRUEBAS_DIARIAS
4,927.0,851.0,76.0
5,1133.0,927.0,206.0
6,1393.0,1133.0,260.0
7,1754.0,1393.0,361.0
8,2343.0,1754.0,589.0
...,...,...,...
178,2625052.0,2599703.0,25349.0
179,2647702.0,2625052.0,22650.0
180,2676154.0,2647702.0,28452.0
181,2699889.0,2676154.0,23735.0


In [57]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=muestras_procesadas.index, y=muestras_procesadas['PRUEBAS_DIARIAS'], mode='lines', line={'color': 'BLUE'}, name='PRUEBAS PROCESADAS DIARIAS'))
fig.show()
